In [17]:
#pool.py
from multiprocessing import Pool
import time

def f(x):
    time.sleep(0.2)
    return x*x

if __name__ == '__main__':
    start_time = time.time()

    arguments = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    # arguments = list(range(20))
    result = []
    for arg in arguments:
        result.append(f(arg))

    print(result)
    end_time = time.time()

    print(f"program run for: {end_time-start_time:.2f}s")


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]
program run for: 4.06s


In [18]:
#noPool.py
import time
def f(x):
    time.sleep(0.2)
    return x*x

if __name__ == '__main__':
    start_time = time.time()

    arguments = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    # arguments = list(range(20))
    result = []
    for arg in arguments:
        result.append(f(arg))

    print(result)
    end_time = time.time()

    print(f"program run for: {end_time-start_time:.2f}s")


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]
program run for: 4.07s


In [19]:
#Processes.py

import multiprocessing as mp
import time
mp.set_start_method('fork')

def f(x):
    time.sleep(0.2)
    print(x*x,end=", ")
    # return x*x

if __name__ == '__main__':
    start_time = time.time()

    arguments = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    # arguments = list(range(20))
    processes = []

    for arg in arguments:
        processes.append(mp.Process(target=f,args=(arg,),))
    for process in processes:
        process.start()
    for process in processes:
        process.join()

    end_time = time.time()

    print(f"program run for: {end_time-start_time:.2f}s")


program run for: 0.64s


In [20]:
#Pipe.py
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'helloFromProcess'])
    conn.send([42, None, 'helloFromProcess1'])
    conn.send([42, None, 'helloFromProcess2'])

def f_read(conn):
    print("other pocess recv: ", conn.recv())
    print("other pocess recv: ", conn.recv())

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p2 = Process(target=f_read, args=(parent_conn,))

    p.start()
    p2.start()
    p2.join()
    p.join()

In [21]:
#que.py
import multiprocessing as mp
import time
def f(q):
    q.put(['hello'])


def f_read(q):
    print(q.get())
def f_read_nowait(q):
    print(q.get_nowait())

if __name__ == '__main__':
    q = mp.Queue()
    p = mp.Process(target=f, args=(q,))
    p2 = mp.Process(target=f_read, args=(q,))
    p3 = mp.Process(target=f_read_nowait, args=(q,))
    # q.put(obj,block,timeout)
    p.start()
    p2.start()
    p3.start()
    p.join()
    p2.join()
    p3.join()


In [22]:
#Shared_mem.py
from multiprocessing import Process, Value, Array

def f(n, a, normal_variable):
    normal_variable.append(1)
    n.value = 123.45
    for i in range(len(a)):
        a[i] = -a[i]

if __name__ == '__main__':
    num = Value('d', 0.0)
    arr = Array('i', range(10))
    normal_variable = [0]
    p = Process(target=f, args=(num, arr, normal_variable))
    p.start()
    p.join()

    print(num.value)
    print(arr[:])
    print(normal_variable)

    f(num, arr, normal_variable)
    print(num.value)
    print(arr[:])
    print(normal_variable)


0.0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0]
123.45
[0, -1, -2, -3, -4, -5, -6, -7, -8, -9]
[0, 1]


In [23]:
#lock.py
from multiprocessing import Process, Lock, Value
import time

def f(l, i, num):
    l.acquire()
    num.value = num.value+1
    time.sleep(0.2)
    try:
        print(f'called : {i}, number of functions at same time:{num.value}')
    finally:
        num.value = num.value-1
        l.release()

def f2(lock,i,num):
    num.value = num.value+1
    time.sleep(0.2)
    print(f'called : {i}, number of functions at same time:{num.value}')
    num.value = num.value-1

if __name__ == '__main__':
    lock = Lock()
    num = Value('i', 0)
    print("concurently:")
    processes=[]
    for i in range(10):
        processes.append(Process(target=f2, args=(lock, i, num)))
        processes[i].start()
    for i in range(10):
        processes[i].join()
    print("sequential:")
    for i in range(10):
        p = Process(target=f2, args=(lock, i, num))
        p.start()
        p.join()
    print("using lock:")
    for i in range(10):
        Process(target=f, args=(lock, i, num)).start()


concurently:
sequential:
using lock:
